### imports

In [ ]:
import pandas as pd
import numpy as np
import gensim
import gensim.downloader as api
import tensorflow as tf
import os

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.datasets import imdb
from keras import preprocessing
from keras.datasets import imdb
from keras.layers import Bidirectional
from keras.models import Model
from keras.layers import Input, LSTM, Dense

from random import shuffle
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Sentiment Analysis

Implement the sentiment analysis of the previous homeworks using a bidirectional LSTM.

### Embedding 50

In [ ]:
def CreateModel():
    #embedding_matrix = get_weights()

    x_i = Input(200) 
    x   = Embedding(1000, 50, 
                    input_length=maxlen,
                    trainable=False)(x_i)
                              

    x = Bidirectional(LSTM(64, return_sequences=True, input_shape=(None,1)))(x)
    x = Dropout(0.2)(x)

    x = Bidirectional(LSTM(32))(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)

  
    x = Dense(1, activation="sigmoid")(x)

    return keras.models.Model(x_i, x)

In [ ]:
# Loading the data
num_words = 10000
maxlen = 200
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

# Creating the model and preprocessing the data
# We pad the sentences here with zeros so it becomes a matrix. 
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)    

model = CreateModel()
model.summary()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 200, 50)           50000     
                                                                 
 bidirectional_8 (Bidirectio  (None, 200, 128)         58880     
 nal)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 200, 128)          0         
                                                                 
 bidirectional_9 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 64)                0   

In [ ]:
# Fitting the model 
model.fit(
    x_train, y_train, epochs=20, batch_size=64,
    validation_split=0.2)

Epoch 1/20
313/313 [==============================] - 19s 44ms/step - loss: 0.6900 - acc: 0.5290 - val_loss: 0.6722 - val_acc: 0.5810
Epoch 2/20
313/313 [==============================] - 12s 40ms/step - loss: 0.6629 - acc: 0.6054 - val_loss: 0.6188 - val_acc: 0.6580
Epoch 3/20
313/313 [==============================] - 12s 40ms/step - loss: 0.6394 - acc: 0.6364 - val_loss: 0.6455 - val_acc: 0.6284
Epoch 4/20
313/313 [==============================] - 13s 40ms/step - loss: 0.6283 - acc: 0.6532 - val_loss: 0.6129 - val_acc: 0.6674
Epoch 5/20
313/313 [==============================] - 12s 40ms/step - loss: 0.6220 - acc: 0.6597 - val_loss: 0.6121 - val_acc: 0.6658
Epoch 6/20
313/313 [==============================] - 13s 41ms/step - loss: 0.6151 - acc: 0.6647 - val_loss: 0.6274 - val_acc: 0.6508
Epoch 7/20
313/313 [==============================] - 12s 40ms/step - loss: 0.6088 - acc: 0.6683 - val_loss: 0.6024 - val_acc: 0.6778
Epoch 8/20
313/313 [==============================] - 14s 43ms

In [ ]:
eval_result = model.evaluate(x_test, y_test)
prediction = model.predict(x_test)
prediction = prediction[:5] 
label = y_test[:5] 

In [ ]:
print(eval_result[1])
print(prediction) 
print(label)

[[0.34699515]
 [0.67189467]
 [0.6448168 ]
 [0.7286762 ]
 [0.9937203 ]]
[0 1 1 0 1]


### W2V Embedding

In [1]:
keyed_vectors = api.load("word2vec-google-news-300")

NameError: ignored

In [ ]:
# Function to prepare embedding layer
def prepare_embedding():
  d = imdb.get_word_index()
  word_index = dict((y,x) for x,y in d.items()) 
  embeddings_index = {}
  
  # This block reads in the embedding from Glove
  f = open(os.path.join('/content/drive/MyDrive/Colab Notebooks/ELEN532_NLP/Lab8', 'glove.6B.50d.txt'))
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close() 

  # Computes embedding matrix
  embedding_matrix= np.zeros((len(word_index), 50))  
  for i, word in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be zeros
        embedding_matrix[i] = embedding_vector

  
  return embedding_matrix 

In [ ]:
def get_weights(key_vec):

    """Loads in w2v embedding and returns weights in the form of a np.array """
  
    # Loading GoogleNews 
    # keyed_vectors = api.load("word2vec-google-news-300")


    # vectors themselves, a 2D numpy array
    weights = key_vec.vectors      

    return weights

In [ ]:
def CreateModel():
    embedding_matrix = get_weights(keyed_vectors)
    # embedding_matrix = prepare_embedding()

    x_i = Input(300) 
    x   = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], 
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False)(x_i)
                              

    x = Bidirectional(LSTM(128, return_sequences=True, input_shape=(None,1)))(x)
    x = Dropout(0.2)(x)

    x = Bidirectional(LSTM(32))(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)

  
    x = Dense(1, activation="sigmoid")(x)

    return keras.models.Model(x_i, x)

In [ ]:
# Loading the data
num_words = 10000
maxlen = 300
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

# Creating the model and preprocessing the data
# We pad the sentences here with zeros so it becomes a matrix. 
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)    

model = CreateModel()
model.summary()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

In [ ]:
# Fitting the model 
model.fit(
    x_train, y_train, epochs=10, batch_size=64,
    validation_split=0.2)

In [ ]:
eval_result = model.evaluate(x_test, y_test)
eval_result[1]

In [ ]:
prediction = model.predict(x_test)
prediction = prediction[:5] 
label = y_test[:5] 
print(prediction) 
print(label)

# 2. Seq2Seq

In this homework, you will create a seq2seq network that __answers questions similar to the questions trained in the training set.__ This dataset is from https://www.kaggle.com/datasets/rtatman/questionanswer-dataset

Your network should use an embedding layer as input.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
filenames = ['S08_question_answer_pairs.txt', 'S09_question_answer_pairs.txt', 'S10_question_answer_pairs.txt']

df = []
for fn in filenames:
    dfi = pd.read_csv(fn, sep='\t',encoding="latin1")
    dfi = dfi[['Question', 'Answer']]
    df.append(dfi)
df = pd.concat(df)
# remove all rows having NAN values
df = df.dropna()
questions = df['Question'].tolist()
# select all answers and put them in a list
answers = df['Answer'].tolist()

In [ ]:
df

,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
1,Was Abraham Lincoln the sixteenth President of...,Yes.
2,Did Lincoln sign the National Banking Act of 1...,yes
3,Did Lincoln sign the National Banking Act of 1...,Yes.
4,Did his mother die of pneumonia?,no
...,...,...
1452,What areas do the Grevy's Zebras inhabit?,semi-arid grasslands of Ethiopia and northern ...
1454,Which species of zebra is known as the common ...,"Plains Zebra (Equus quagga, formerly Equus bur..."
1455,Which species of zebra is known as the common ...,Plains Zebra
1456,At what age can a zebra breed?,five or six


In [ ]:
df.iloc[2074]

Question                                    What is Santiago?
Answer      Santiago is Spanish for St. James. It is also ...
Name: 665, dtype: object

In [ ]:
df = df.drop(df.index[2074])
df = df[df.Answer != "no"]
df = df[df.Answer != "Yes."]

In [ ]:
for txt in target_texts:
  if len(txt) ==  max([len(txt) for txt in target_texts]):
    print(target_texts.index(txt))

140
425


In [ ]:
df["Answer"] 

1                                                   Yes.
3                                                   Yes.
5                                                    No.
6                                              18 months
7                                             18 months.
                             ...                        
814                                        from G3 to C8
815                                        from G3 to C8
819                                            Indonesia
820    Arnold Schlick's Spiegel der Orgelmacher und O...
821            by a bright, sharp tone and high register
Name: Answer, Length: 1432, dtype: object

In [ ]:
batch_size = 64 
epochs = 100
latent_dim = 256

In [ ]:
# Vectorize data 

input_texts = [] # list of all "Questions"
target_texts = [] # list of all corresponding "Answers"
input_char = set() # list of unique letters, tokenized letters (includes numbers, punctuations)
target_char = set() # list of unique letters 


for line in range(len(df["Question"])):
  input_text = df["Question"].iloc[line]
  #if type(input_text) != str: # Converting all texts in questions column to str type
    #input_text = str(input_text)
  input_texts.append(input_text)

  target_text = df["Answer"].iloc[line]
  #if type(target_text) != str: # Converting all texts in answer column to str type
    #target_text = str(target_text)
  target_text = '\t' + target_text + '\n'
  target_texts.append(target_text)


  for char in input_text:
      if char not in input_char:
          input_char.add(char)
  for char in target_text:
      if char not in target_char:
          target_char.add(char)

input_char = sorted(list(input_char))
target_char = sorted(list(target_char))
  
# Counting the number of tokens after sorting 
num_encoder_tokens = len(input_char)
num_decoder_tokens = len(target_char)

# Finds the longest sentence in input and target texts list 
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


# Create a dictionary for input & target chracter tokens 
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_char)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_char)])


# Create numpy array of for encoder, decoder input data & decoder target data
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# "input_text" refers to a single element in list "input_texts". Same for target text
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens)) # input is the number of encoder tokens 
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences, and to return internal states as well. 
# We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Training the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
40/40 [==============================] - 8s 119ms/step - loss: 0.2390 - val_loss: 0.3423
Epoch 2/100
40/40 [==============================] - 4s 92ms/step - loss: 0.2341 - val_loss: 0.3427
Epoch 3/100
40/40 [==============================] - 4s 92ms/step - loss: 0.2327 - val_loss: 0.3362
Epoch 4/100
40/40 [==============================] - 4s 93ms/step - loss: 0.2311 - val_loss: 0.3341
Epoch 5/100
40/40 [==============================] - 4s 92ms/step - loss: 0.2296 - val_loss: 0.3322
Epoch 6/100
40/40 [==============================] - 4s 93ms/step - loss: 0.2284 - val_loss: 0.3304
Epoch 7/100
40/40 [==============================] - 4s 92ms/step - loss: 0.2273 - val_loss: 0.3255
Epoch 8/100
40/40 [==============================] - 4s 92ms/step - loss: 0.2256 - val_loss: 0.3264
Epoch 9/100
40/40 [==============================] - 4s 92ms/step - loss: 0.2251 - val_loss: 0.3237
Epoch 10/100
40/40 [==============================] - 4s 92ms/step - loss: 0.2241 - val_loss: 0.324

In [ ]:
# INFERENCE MODE

# Decoding test sentence STEPS:
# 1) Encode the input sentence and retrieve the initial decoder state
# 2) Run one step of the decoder with this initial state and a "start of sequence" token as target. The output will be the next target character.
# 3) Append the target character predicted and repeat.

# Inference setup 
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Was Abraham Lincoln the sixteenth President of the United States?
Decoded sentence: Yes

-
Input sentence: Did Lincoln sign the National Banking Act of 1863?
Decoded sentence: Yes

-
Input sentence: Did his mother die of pneumonia?
Decoded sentence: Yes

-
Input sentence: How many long was Lincoln's formal education?
Decoded sentence: Yes

-
Input sentence: How many long was Lincoln's formal education?
Decoded sentence: Yes

-
Input sentence: When did Lincoln begin his political career?
Decoded sentence: Yes

-
Input sentence: When did Lincoln begin his political career?
Decoded sentence: Yes

-
Input sentence: What did The Legal Tender Act of 1862 establish?
Decoded sentence: Yes

-
Input sentence: What did The Legal Tender Act of 1862 establish?
Decoded sentence: Yes

-
Input sentence: Who suggested Lincoln grow a beard?
Decoded sentence: Yes

-
Input sentence: Who suggested Lincoln grow a beard?
Decoded sentence: Yes

-
Input sentence: When did the Gettysburg addre